In [15]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Análise de rentabilidade

Este documento replica a planilha utilizada para realizar a avaliação de rentabilidade de um paciente. O objetivo é que esta seja a base para podermos automatizar o processo. 

## Informações

Na planilha, a primeira página que merece atenção é a de "Informações". Aqui é calculado um custo fixo do paciente com base em informações conhecidas anteriormente. Abaixo temos um dicionário com todos os serviços presentes e seus respectivos valores base. Nessa página, ainda se é calculado o custo fixo total do paciente, também presente na célula abaixo. 

In [16]:
# custo fixo de cada serviço
tipos_serviço = {
    "Internação Domiciliar 6hrs": 50,
    "Internação Domiciliar 12hrs": 100,
    "Internação Domiciliar 24hrs": 200,
    "Assistência Domiciliar": 35,
    "Terapias": 9,
    "Curabem": 28.8,
    "Oxigenoterapia": 11.6,
    "Fiqbem": 3.10
}

# cálculo do custo fixo total do paciente
def custo_fixo_total(tipo_serviço: str, periodo: int):
    servico_custo = tipos_serviço[tipo_serviço]
    custo_total = servico_custo * periodo
    return custo_total

## Dados

A seguir, os dados do orçamento são inseridos na planilha a partir de um Excel ou CSV. Aqui, temos um paciente de exemplo que vem de um CSV fornecido por Thiago. 

Aqui, carregamos os dados em formato de arquivo Python, de uma forma em que cada objeto representa uma linha e cada atributo desse objeto representa uma coluna. 

In [17]:
import dados_planilha

dados_planilha.orcamento[0]

{'descricao': 'DIARIA HC - ALTA COMPLEXIDADE',
 'quantidade': 20.0,
 'unidade': 'UN',
 'valor_unitario': '785,00',
 'valor_total': '15.700,00',
 'grupo_procedimento': 'DIARIAS',
 'custo_unitario': '316,63',
 'custo_total': '6.332,60',
 'pacote': 'NÃO'}

## Rentabilidade

Após o carregamento dos dados, a rentabilidade é gerada. Para isso, alguns procedimentos são realizados. A principio, se separa o que será pago por pacote ou não, isto é, quais itens geram receita ou não. 

* Se um item é marcado como "sim" na coluna PACOTE, isso significa que ele está contido no pacote cobrado ao convênio. 
* Se um item é marcado como "não" na coluna PACOTE, isso indica que ele não está incluso no pacote e será cobrado ao convênio de forma a parte. Este valor entra no cálculo da receita bruta, pois representa o dinheiro que a empresa irá receber pelo item específico.

Caso o paciente não seja por pacote, o convênio tende a pagar por cada item ou pelo menos a maioria deles, mas isso nem sempre é o caso.

Pelos dados que vem do CSV, também não temos definidos quais itens são parte do pacote ou não. Hoje, isso é tirado "da cabeça de alguém", e não está 100% mapeado internamente. Hoje temos contratos para cada operadora, e cada contrato tem detalhes específicos de como funcionam as regras de cada item. Por exemplo, há uma operadora onde remédios acima de 300 reais não são inclusos no pacote (são solicitados para a operadora a parte), enquanto que em outras essa restrição não acontece. O paciente é dividido entre baixa, média e alta complexidade. Cada operadora tem sua regra de negócio.

Precisamos definir como isso é determinado de maneira programática. Por enquanto, vamos considerar que se for por pacote, é por que há itens marcados com ```"grupo_procedimento" = "DIARIAS"``` nos dados. Caso contrário, entenderemos que todos os itens serão pagos pelo convênio.


In [18]:
# Verifica se o paciente é por pacote
def checar_paciente_pacote(itens_orcamento: list):
    for i in itens_orcamento:
        if i["grupo_procedimento"] == "DIARIAS":
            return True
    return False

# Trata o paciente para o caso em que ele é pacote
# TODO: ISTO PRECISA SER VALIDADO COM THIAGO
def paciente_pacote(itens_orcamento):
    for i in itens_orcamento:
        if i["grupo_procedimento"] == "DIARIAS":
            i["pacote"] = "NÃO"
        else: i["pacote"] = "SIM"
    return itens_orcamento

# Trata o paciente para o caso em que ele não é pacote
# TODO: ISTO PRECISA SER VALIDADO COM THIAGO
def paciente_nao_pacote(itens_orcamento):
    for i in itens_orcamento:
        i["pacote"] = "NÃO"
    return itens_orcamento

pacote = checar_paciente_pacote(dados_planilha.orcamento)

if pacote: itens_orcamento = paciente_pacote(dados_planilha.orcamento)
else: itens_orcamento = paciente_nao_pacote(dados_planilha.orcamento)

itens_orcamento[0]

{'descricao': 'DIARIA HC - ALTA COMPLEXIDADE',
 'quantidade': 20.0,
 'unidade': 'UN',
 'valor_unitario': '785,00',
 'valor_total': '15.700,00',
 'grupo_procedimento': 'DIARIAS',
 'custo_unitario': '316,63',
 'custo_total': '6.332,60',
 'pacote': 'NÃO'}

### Receita bruta e líquida

A receita bruta é calculada ao somar todas as fontes de receita. Isso inclui as diárias, mas também qualquer outro item marcado como ```pacote = não```. Aqui calculamos individualmente o valor de cada categoria salvamos o total. A receita líquida é calculada a partir disso, após descontar impostos que totalizam 5,65%.  

In [19]:

# categorias
grupo_procedimentos = [
    "DIARIAS",
    "TAXAS",
    "SERV ENFERMAGEM",
    "SERVICOS DE FISIOTERAPIA",
    "SERVICOS DE FONOAUDIOLOGIA",
    "SERVICOS DE NUTRICAO",
    "SERVICOS DE TERAPIA OCUPACIONAL",
    "VISITAS E CONSULTAS MEDICAS - HC",
    "MEDICAMENTOS",
    "MEDICAMENTOS RESTRITO HOSP.",
    "ALIMENTACOES E DIETAS",
    "MATERIAIS",
    "CURATIVOS ESPECIAIS"
]

df = pd.DataFrame(grupo_procedimentos, columns=["Grupo de Procedimento"])

import re
# função auxiliar para o parsing dos valores
def parse_valor(s):
    if s is None:
        return 0.0
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    s = re.sub(r'[^\d,.\-()]', '', s)
    negativo = False
    if s.startswith('(') and s.endswith(')'):
        negativo = True
        s = s[1:-1]
    s = s.replace('.', '').replace(',', '.')
    try:
        val = float(s) if s not in ('', '.', '-') else 0.0
    except ValueError:
        val = 0.0
    return -val if negativo else val

# calculo da receita bruta de uma categoria.
def receita_bruta_categoria(itens_orcamento, categoria):
    total = 0.0
    for item in itens_orcamento:
        if item["grupo_procedimento"] == categoria and item["pacote"] == "NÃO":
            total += parse_valor(item["valor_total"])
    return total

TOTAL_RECEITA_BRUTA = 0
TAXA_TOTAL = 0.02 + 0.0065 + 0.03   # 5,65%
FATOR_LIQUIDO = 1 - TAXA_TOTAL

# somando a receita bruta de cada categoria
items_gross = []
items_liquid = []
for categoria in grupo_procedimentos:
    rect_bruta = receita_bruta_categoria(itens_orcamento, categoria)
    items_gross.append(rect_bruta)
    receita_liquida = rect_bruta * FATOR_LIQUIDO
    items_liquid.append(receita_liquida)
    TOTAL_RECEITA_BRUTA += rect_bruta
df["Receita Bruta"] = items_gross
df["Receita Líquida"] = items_liquid
print("============RECEITA BRUTA TOTAL============")
print(TOTAL_RECEITA_BRUTA)

receita_liquida = TOTAL_RECEITA_BRUTA * FATOR_LIQUIDO
print("============RECEITA LÍQUIDA TOTAL============")
print(receita_liquida)
df

============RECEITA BRUTA TOTAL============
15700.0
============RECEITA LÍQUIDA TOTAL============
14812.95


,Grupo de Procedimento,Receita Bruta,Receita Líquida
0,DIARIAS,15700.0,14812.95
1,TAXAS,0.0,0.00
2,SERV ENFERMAGEM,0.0,0.00
3,SERVICOS DE FISIOTERAPIA,0.0,0.00
4,SERVICOS DE FONOAUDIOLOGIA,0.0,0.00
5,SERVICOS DE NUTRICAO,0.0,0.00
6,SERVICOS DE TERAPIA OCUPACIONAL,0.0,0.00
7,VISITAS E CONSULTAS MEDICAS - HC,0.0,0.00
8,MEDICAMENTOS,0.0,0.00
9,MEDICAMENTOS RESTRITO HOSP.,0.0,0.00


## Custos

É feito agora um calculo similar para os custos dos itens do orçamento. Somamos o custo total de todos os itens de cada categoria, e depois somamos tudo para obter o custo total do paciente. Além disso, temos os custos fixos do paciente, que são calculados com base na aba de informações da planilha e dependem da complexidade do paciente. Nesse caso, complexidade afeta apenas pacientes de Internação Domiciliar (ID) e não Assistência Domiciliar (AD).

In [20]:
# Cada complexidade de paciente é mapeada para um tipo de serviço com custo fixo específico
map_custos = {
    "BAIXA COMPLEXIDADE": "Internação Domiciliar 6hrs",
    "MEDIA COMPLEXIDADE": "Internação Domiciliar 12hrs",
    "ALTA COMPLEXIDADE": "Internação Domiciliar 24hrs"
}

# calcula o custo total de cada categoria de custo
def custos_mv(itens_orcamento, categoria):
    total = 0.0
    for item in itens_orcamento:
        if item["grupo_procedimento"] == categoria:
            total += parse_valor(item["custo_total"])
    return total

TOTAL_CUSTO_MV = 0

# soma o custo de todas as categorias
items_custo = []
for categoria in grupo_procedimentos:
    custos = custos_mv(itens_orcamento, categoria)
    items_custo.append(custos)
    TOTAL_CUSTO_MV += custos

df["Custos MV"] = items_custo
print("============CUSTOS TOTAIS DO MV============")
print(TOTAL_CUSTO_MV)
print("============CUSTOS FIXOS============")

# calcula o custo fixo total com base na complexidade e na quantidade de dias internado do paciente
df["Custos Fixos"] = 0.0
c_fixo = custo_fixo_total(map_custos[dados_planilha.cabecalho["internacao"]["complexidade"]], int(dados_planilha.cabecalho["internacao"]["quantidade_dias"])) * 1.1
df.loc[df['Grupo de Procedimento'] == 'TAXAS', 'Custos Fixos'] = c_fixo
df

============CUSTOS TOTAIS DO MV============
11889.920000000002
============CUSTOS FIXOS============


,Grupo de Procedimento,Receita Bruta,Receita Líquida,Custos MV,Custos Fixos
0,DIARIAS,15700.0,14812.95,6332.60,0.0
1,TAXAS,0.0,0.00,1419.66,4400.0
2,SERV ENFERMAGEM,0.0,0.00,50.00,0.0
3,SERVICOS DE FISIOTERAPIA,0.0,0.00,357.36,0.0
4,SERVICOS DE FONOAUDIOLOGIA,0.0,0.00,190.24,0.0
5,SERVICOS DE NUTRICAO,0.0,0.00,84.14,0.0
6,SERVICOS DE TERAPIA OCUPACIONAL,0.0,0.00,322.50,0.0
7,VISITAS E CONSULTAS MEDICAS - HC,0.0,0.00,200.00,0.0
8,MEDICAMENTOS,0.0,0.00,138.33,0.0
9,MEDICAMENTOS RESTRITO HOSP.,0.0,0.00,558.03,0.0


## Margens e rentabilidade

Para calcular a margem bruta de cada categoria, subtraimos os custos da receita líquida. Depois, calculamos os impostos, que na planilha é uma simples subtração de 34%. Após isso, subtraimos os impostos da margem bruta para obter a margem líquida. Por fim, dividimos a margem líquida pela receita líquida para obter a rentabilidade.

In [21]:
import numpy as np

df["Margem Bruta"] = df["Receita Líquida"] - (df["Custos MV"] + df["Custos Fixos"])

margem_bruta = receita_liquida - (TOTAL_CUSTO_MV + c_fixo)
margem_bruta_positiva = df["Margem Bruta"] > 0
margem_bruta_positiva_total = df.loc[margem_bruta_positiva, "Margem Bruta"].sum()

impostos =  margem_bruta * 0.34 if margem_bruta > 0 else 0

if margem_bruta_positiva_total > 0:
    taxa_rateio = impostos / margem_bruta_positiva_total
else:
    taxa_rateio = 0

df["Impostos"] = np.where(
    df["Margem Bruta"] > 0,
    df["Margem Bruta"] * taxa_rateio,    
    0
)

margem_liquida = margem_bruta - impostos
df["Margem Líquida"] = df["Margem Bruta"] - df["Impostos"]

print("============RENTABILIDADE============")
margem_liquida = margem_bruta - impostos
print(str(round((margem_liquida / receita_liquida) * 100, 2)) + "%")

totais = ["TOTAL", TOTAL_RECEITA_BRUTA, receita_liquida, TOTAL_CUSTO_MV, c_fixo, margem_bruta, impostos, margem_liquida]
df.loc[len(df)] = totais
df

============RENTABILIDADE============
-9.97%


,Grupo de Procedimento,Receita Bruta,Receita Líquida,Custos MV,Custos Fixos,Margem Bruta,Impostos,Margem Líquida
0,DIARIAS,15700.0,14812.95,6332.60,0.0,8480.35,0.0,8480.35
1,TAXAS,0.0,0.00,1419.66,4400.0,-5819.66,0.0,-5819.66
2,SERV ENFERMAGEM,0.0,0.00,50.00,0.0,-50.00,0.0,-50.00
3,SERVICOS DE FISIOTERAPIA,0.0,0.00,357.36,0.0,-357.36,0.0,-357.36
4,SERVICOS DE FONOAUDIOLOGIA,0.0,0.00,190.24,0.0,-190.24,0.0,-190.24
5,SERVICOS DE NUTRICAO,0.0,0.00,84.14,0.0,-84.14,0.0,-84.14
6,SERVICOS DE TERAPIA OCUPACIONAL,0.0,0.00,322.50,0.0,-322.50,0.0,-322.50
7,VISITAS E CONSULTAS MEDICAS - HC,0.0,0.00,200.00,0.0,-200.00,0.0,-200.00
8,MEDICAMENTOS,0.0,0.00,138.33,0.0,-138.33,0.0,-138.33
9,MEDICAMENTOS RESTRITO HOSP.,0.0,0.00,558.03,0.0,-558.03,0.0,-558.03
